In [ ]:
# ЭТО ВСЕ ТЕСТЫ! ЗА АКТУАЛЬНОЙ ВЕРСИЕЙ - В upd_mn.pyw

In [2]:
import pandas as pd

In [7]:
df = pd.read_csv(r"C:\Users\igor.i.plotnikov\PycharmProjects\work\MN\2021-06\Outgoing MN direct daily за весь прошлый месяц Июнь.csv",
    sep=';', delimiter=None, engine='python')
df.head()

,CDR_Date,Branch,GlobalCarrier,Destination_Band,Minutes,Cost,Tariff,Rate_List_Currency1
0,01.06.2021,DZ BRN,T_INT,Азия. Монголия,"0,05","0,04","0,79",Rouble
1,01.06.2021,DZ BRN,T_INT,"Азия. Монголия, Моб.","2,68","2,12","0,79",Rouble
2,01.06.2021,DZ BRN,T_INT,Европа. Турция моб.Aycell,"0,05","0,58","11,55",Rouble
3,01.06.2021,DZ BRN,T_INT,СНГ. Азербайджан моб.Azercell,"16,98","290,42","17,1",Rouble
4,01.06.2021,DZ BRN,T_INT,СНГ. Казахстан,"7,53","15,44","2,05",Rouble


In [20]:
df['Cost'].dtype == 'O'

True

In [11]:
df['Minutes'] = df['Minutes'].apply(lambda x: pd.to_numeric(x.str.replace(',', '.'), downcast='float'))

AttributeError: 'float' object has no attribute 'str'

In [10]:
df.head()

,CDR_Date,Branch,GlobalCarrier,Destination_Band,Minutes,Cost,Tariff,Rate_List_Currency1
0,01.06.2021,DZ BRN,T_INT,Азия. Монголия,0.05,"0,04","0,79",Rouble
1,01.06.2021,DZ BRN,T_INT,"Азия. Монголия, Моб.",2.68,"2,12","0,79",Rouble
2,01.06.2021,DZ BRN,T_INT,Европа. Турция моб.Aycell,0.05,"0,58","11,55",Rouble
3,01.06.2021,DZ BRN,T_INT,СНГ. Азербайджан моб.Azercell,16.98,"290,42","17,1",Rouble
4,01.06.2021,DZ BRN,T_INT,СНГ. Казахстан,7.53,"15,44","2,05",Rouble


In [62]:
# CREATE MULTISET TABLE UAT_PRODUCT.MN_DIRECT_IN_OUT ,NO FALLBACK ,
#      NO BEFORE JOURNAL,
#      NO AFTER JOURNAL,
#      CHECKSUM = DEFAULT,
#      DEFAULT MERGEBLOCKRATIO,
#      MAP = TD_MAP1
#      (
#       CDR_MONTH DATE FORMAT 'YYYY-MM-DD',
#       CDR_DATE DATE FORMAT 'YYYY-MM-DD',
#       CARRIER VARCHAR(50) CHARACTER SET UNICODE NOT CASESPECIFIC,
#       BRANCH VARCHAR(50) CHARACTER SET UNICODE NOT CASESPECIFIC,
#       DIRECTION VARCHAR(50) CHARACTER SET UNICODE NOT CASESPECIFIC,
#       DESTINATION VARCHAR(50) CHARACTER SET UNICODE NOT CASESPECIFIC,
#       MINUTES DECIMAL(18,6),
#       COST DECIMAL(18,6),
#       CURRENCY VARCHAR(50) CHARACTER SET UNICODE NOT CASESPECIFIC,
#       INSERT_DATE DATE FORMAT 'YYYY-MM-DD')
# PRIMARY INDEX ( CDR_MONTH ,CDR_DATE ,CARRIER ,BRANCH ,DIRECTION ,
# DESTINATION );


# скрипт идет в почту
# проверяет письма за {2} последних дня
# для каждого письма: если отправитель Interconnect Reporting System и есть вложение .csv,
#     то создаем объект класса Handler, прокидываем в него содержимое файла и его название
# определяем, что это за отчет, смотрим максимальную дату для него. если она превышает максимальную дату в базе для этого отчета,
#     то удаляем в базе текущий месяц
#     заливаем в базу текущий месяц из файла
#     сохраняем актуальный файл в папку \Data


from exchangelib import Credentials, Account, FileAttachment, EWSDateTime, EWSTimeZone
import os.path
import pandas as pd
import teradatasql
from datetime import datetime, timedelta
import configparser
import numpy as np
import io
import sys


class Handler():

    tablename = 'UAT_PRODUCT.MN_DIRECT_IN_OUT'
    cols_dict = {'CDRDate1': 'CDR_DATE',
                   'Carrier1': 'CARRIER',
                   'Traffic_Direction21': 'DIRECTION',
                   'Destination1': 'DESTINATION',
                   'TotalDurationInMinutes_Sum': 'MINUTES',
                   'Total_Calls_Connected': 'TOTAL_CALLS_CONNECTED',
                   'Total_Cost': 'COST',
                   'Rate_List_Currency1': 'CURRENCY',
                   'CDR_Date': 'CDR_DATE',
                   'GlobalCarrier': 'CARRIER',
                   'Branch': 'BRANCH',
                   'Destination_Band': 'DESTINATION',
                   'Minutes': 'MINUTES',
                   'Cost': 'COST',
                   'CDRDate_Value': 'CDR_DATE',
                   'GlobalCarrier_Value': 'CARRIER',
                   'Region_Value': 'BRANCH',
                   'Destination_Value': 'DESTINATION',
                   'TotalDurationInMinutes_Value': 'MINUTES',
                   }
     
    def __init__(self, csv_content, csv_name):
        self.csv_name = csv_name
        self.direction = self.csv_name.split(' ')[0]
        self.carrier_type = ('H' if 'H_' in self.csv_name else 'T')
        self.scheme, self.table = self.tablename.split('.')
        self.raw = pd.read_csv(csv_content, thousands=',')
        self.df = pd.DataFrame()
        self.report_month = None
        self.max_csv_date = None
        self.max_db_date = None
        


    def process(self):
        print(f'Processing {self.csv_name}...')
        df = self.raw.copy()
        df.rename(self.cols_dict, axis=1, inplace=True)
        if 'BRANCH' not in df.columns:
            df[['CARRIER','BRANCH']] = df['CARRIER'].str.extract(r'([A-Za-z0-9_]*)\s(T2\s[A-Za-z0-9]*)',expand=True,)
        df['CARRIER_TYPE'] = df['CARRIER'].str.extract(r'([HT])\_[A-Z0-9]*',expand=True,)
        df = df[df['CARRIER_TYPE'] == self.carrier_type].reset_index()
        df['DIRECTION'] = self.direction
        df['MINUTES'] = df['MINUTES'].apply(lambda x: pd.to_numeric(x, downcast='float'))
        df['INSERT_DATE'] = datetime.today().strftime('%Y-%m-%d')
        
        
        for date_format in ('%d.%m.%Y', '%d.%m.%y', '%d/%m/%Y'):
            try:
                df['CDR_DATE'] = df['CDR_DATE'].apply(lambda x: datetime.strptime(x, date_format).strftime('%Y-%m-%d'))
            except ValueError:
                continue
            else:
                df['CDR_MONTH'] = self.report_month = df['CDR_DATE'][0][0:-2] + '01'
                self.max_csv_date = max(df['CDR_DATE'])
                break
        try:
            db_columns = self.get_db_columns()
        except:
            print(f'Error: columns list is not available for {self.tablename}')
            sys.exit(0)
        else:
            df = df.reindex(columns=db_columns, fill_value=np.NaN).where((pd.notnull(df)), None)
        self.df = df
        print(f'Data is processed.')


    def get_db_columns(self):
        with teradatasql.connect() as con:
            with con.cursor() as cur:
                cur.execute(f''' 
                    SEL rtrim(ColumnName) as ColumnName
                    FROM  DBC.Columns
                    WHERE DatabaseName = '{self.scheme}'
                        and TableName = '{self.table}'
                    order by ColumnId

                ''')
                db_columns = [col[0] for col in cur.fetchall()]
        return db_columns


    def get_max_db_date(self):
        print(f'Checking for relevance...')
        with teradatasql.connect() as con:
            with con.cursor() as cur:
                cur.execute(f'''
                    SELECT
                        MAX(CDR_DATE)
                    FROM {self.tablename}
                    WHERE DIRECTION = '{self.direction}'
                        AND CDR_MONTH = '{self.report_month}'
                        AND CARRIER_TYPE = '{self.carrier_type}'
                ''')
                max_db_date = cur.fetchall()[0][0]
        if max_db_date:
            return max_db_date.strftime('%Y-%m-%d')
        else:
            return self.report_month


    def check_for_insert(self):
        self.max_db_date = self.get_max_db_date()
        if not self.max_db_date:
            print('Error: max_db_date is not defined')
        elif not self.max_csv_date:
            print('Error: max_csv_date is not defined.')
        elif self.max_csv_date >= self.max_db_date:
            self.insert()
            self.filename = f'{sys.path[0]}\\data\\{self.csv_name} {self.report_month}.xlsx'
            self.df.to_excel(self.filename, index=False)
            print(f'Saved as {self.filename}.\n')
        elif self.max_csv_date < self.max_db_date:
            print('The report is outdated.\n')


    def insert(self, batchsize=100000, replace=True):
        rows_count, cols_count = self.df.shape
        with teradatasql.connect() as con:
            with con.cursor() as cur:
                if replace:
                    print(f'Deleting from {self.tablename}...')
                    cur.execute(f'''
                        DELETE FROM {self.tablename}
                        WHERE DIRECTION = '{self.direction}'
                            AND CDR_MONTH = '{self.report_month}'
                            AND CARRIER_TYPE = '{self.carrier_type}'
                    ''')
                    print(f'{cur.rowcount} rows deleted from {self.tablename}.')

                print(f'Inserting into {self.tablename}...')
                for num in range(0, rows_count, batchsize):
                    cur.executemany(f'''
                         INSERT into {self.tablename} ({','.join('?' * cols_count)})
                        ''', [tuple(row) for row in self.df.iloc[num:num + batchsize, :].itertuples(index=False)]
                                    )
                print(f'{rows_count} rows inserted into {self.tablename} in total.')

                print(f'Collecting statistics...')
                cur.execute(f'''
                    SELECT rtrim(ColumnName) as ColumnName
                    FROM  dbc.indices
                    where DatabaseName = '{self.scheme}'
                        and TableName = '{self.table}'
                ''')
                columns = cur.fetchall()
                col_name = [f'COLUMN({col[0]})' for col in columns]
                collect_query = 'COLLECT STATISTICS ' + ', '.join(col_name) + f' ON {self.tablename};'
                cur.execute(collect_query)
                print(f'Statistics were collected.')



def log_into_account(mailbox: str):
    print('Logging in...\n')
    c = configparser.ConfigParser()
    c.read('C:\\Users\\igor.i.plotnikov\\setup.cfg')
    user = c['teradata']['user']
    pw = c['teradata']['password']
    # https://webmail.tele2.ru/owa/
    credentials = Credentials(username=user, password=pw)
    account = Account(mailbox, credentials=credentials, autodiscover=True)
    return account


def get_messages(account, number_of_days: int):
    print('Checking email...\n')
    tz = EWSTimeZone.localzone()
    now = EWSDateTime.now().replace(tzinfo=tz)
    period = now - timedelta(days=number_of_days)
    messages = account.inbox.filter(datetime_received__gt=period).order_by('-datetime_received')
    return messages


def main(source_type: str, source: str):
    if source_type == 'mailbox':
        account = log_into_account(mailbox=source)   # 'igor.i.plotnikov@tele2.ru'
        last_msgs = get_messages(account, number_of_days=1)
        for msg in last_msgs:
            if msg.sender.name == 'Interconnect Reporting System':
                for attachment in msg.attachments:
                    if isinstance(attachment, FileAttachment) and attachment.name.endswith('.csv'):
                        msg_datetime_str = msg.datetime_received.replace(tzinfo=None).strftime("%Y-%m-%d %H-%M-%S")
                        print(f'Importing {attachment.name} received at {msg_datetime_str}...')
                        csv_report = io.StringIO(attachment.content.decode('utf-8'))
                        try:
                            handler = Handler(csv_report, attachment.name)
                        except:
                            print(f'Report version is not defined for {attachment.name} received at {msg_datetime_str}.')
                        else:
                            handler.process()
                            handler.check_for_insert()
    elif source_type == 'directory':
        if source[-1] != '\\': source = source + '\\'
        for file in os.listdir(source):
            if file.endswith('.csv'):
                filepath = source + file
                print(f'Importing {filepath}...')
                try:
                    handler = Handler(filepath, file)
                except:
                    print(f'Report version is not defined for {filepath}')
                else:
                    handler.process()
                    handler.check_for_insert()
    print('\nAll IC data is up to date.\n')
    return None


# if __name__ == '__main__':
#     main(source_type='directory', source=r'C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\2021-03')
#     main(source_type='mailbox', source='product_reporting@tele2.ru', versions_list=versions_list)

In [64]:
main(source_type='mailbox', source='product_reporting@tele2.ru')

Logging in...

Checking email...

Importing Outgoing MN direct daily.csv received at 2021-04-29 07-53-23...
Processing Outgoing MN direct daily.csv...
Data is processed.
Checking for relevance...
Deleting from UAT_PRODUCT.MN_DIRECT_IN_OUT...
119662 rows deleted from UAT_PRODUCT.MN_DIRECT_IN_OUT.
Inserting into UAT_PRODUCT.MN_DIRECT_IN_OUT...
119662 rows inserted into UAT_PRODUCT.MN_DIRECT_IN_OUT in total.
Statistics were collected.
Saved as C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\data\Outgoing MN direct daily.csv 2021-04-01.xlsx.

Importing Incoming MN daily.csv received at 2021-04-29 07-42-58...
Processing Incoming MN daily.csv...
Data is processed.
Checking for relevance...
Deleting from UAT_PRODUCT.MN_DIRECT_IN_OUT...
22906 rows deleted from UAT_PRODUCT.MN_DIRECT_IN_OUT.
Inserting into UAT_PRODUCT.MN_DIRECT_IN_OUT...
29354 rows inserted into UAT_PRODUCT.MN_DIRECT_IN_OUT in total.
Statistics were collected.
Saved as C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\data\Inc

In [65]:
main(source_type='directory', source=r'C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\2021-03')

Importing C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\2021-03\Incoming MN daily за весь прошлый месяц.csv...
Processing Incoming MN daily за весь прошлый месяц.csv...
Data is processed.
Checking for relevance...
Deleting from UAT_PRODUCT.MN_DIRECT_IN_OUT...
0 rows deleted from UAT_PRODUCT.MN_DIRECT_IN_OUT.
Inserting into UAT_PRODUCT.MN_DIRECT_IN_OUT...
36549 rows inserted into UAT_PRODUCT.MN_DIRECT_IN_OUT in total.
Statistics were collected.
Saved as C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\data\Incoming MN daily за весь прошлый месяц.csv 2021-03-01.xlsx.

Importing C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\2021-03\Incoming MN direct H_carrier daily за весь прошлый месяц.csv...
Processing Incoming MN direct H_carrier daily за весь прошлый месяц.csv...
Data is processed.
Checking for relevance...
Deleting from UAT_PRODUCT.MN_DIRECT_IN_OUT...
0 rows deleted from UAT_PRODUCT.MN_DIRECT_IN_OUT.
Inserting into UAT_PRODUCT.MN_DIRECT_IN_OUT...
41288 rows inserted into UA

In [53]:
handler = Handler(r'C:\Users\igor.i.plotnikov\PycharmProjects\work\ИК\2021-03\Incoming MN direct H_carrier daily за весь прошлый месяц.csv',
                  'Incoming MN direct H_carrier daily за весь прошлый месяц.csv')
handler.process()
handler.df.head()

Processing Incoming MN direct H_carrier daily за весь прошлый месяц.csv...
Data is processed.


,CDR_MONTH,CDR_DATE,CARRIER_TYPE,CARRIER,BRANCH,DIRECTION,DESTINATION,MINUTES,COST,CURRENCY,INSERT_DATE
0,2021-03-01,2021-03-01,None,H_A1T,T2 CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to CHE,4.3,0.5461,Euro,2021-04-29
1,2021-03-01,2021-03-01,None,H_A1T,T2 CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to IZH,2.47,0.31327,Euro,2021-04-29
2,2021-03-01,2021-03-01,None,H_A1T,T2 CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to KAZ,1.22,0.15452,Euro,2021-04-29
3,2021-03-01,2021-03-01,None,H_A1T,T2 CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to KRG,0.4,0.0508,Euro,2021-04-29
4,2021-03-01,2021-03-01,None,H_A1T,T2 CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to KRS,5.33,0.67733,Euro,2021-04-29


In [27]:
# handler.df[['CARRIER','BRANCH']] = handler.df['CARRIER'].str.split(' T2 ',expand=True,)
# handler.df.head()

,CDR_MONTH,CDR_DATE,CARRIER,BRANCH,DIRECTION,DESTINATION,MINUTES,COST,CURRENCY,INSERT_DATE
0,2021-03-01,2021-03-01,H_A1T,CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to CHE,4.3,0.5461,Euro,2021-04-29
1,2021-03-01,2021-03-01,H_A1T,CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to IZH,2.47,0.31327,Euro,2021-04-29
2,2021-03-01,2021-03-01,H_A1T,CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to KAZ,1.22,0.15452,Euro,2021-04-29
3,2021-03-01,2021-03-01,H_A1T,CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to KRG,0.4,0.0508,Euro,2021-04-29
4,2021-03-01,2021-03-01,H_A1T,CFN,Incoming,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to KRS,5.33,0.67733,Euro,2021-04-29


In [ ]:
# with teradatasql.connect() as con:
#     with con.cursor() as cur:
#         cur.execute(f'''
#             SELECT
#                 MAX(CDR_DATE)
#             FROM UAT_PRODUCT.MN_DIRECT_H_CARRIER_IN_OUT
#         ''')
#         max_db_date = cur.fetchall()[0][0].strftime('%Y-%m-%d')
# if max_db_date:
#     return max_db_date.strftime('%Y-%m-%d')

In [ ]:

# try:
#     inserted_files = pickle.load(open('inserted_files.p', "rb"))
# except EOFError:
#     inserted_files = list()

In [ ]:
'DEST_GLOBAL'
'DEST_COUNTRY'
'DEST_LOCAL'

In [299]:
df['DEST_FROM'] = df['DESTINATION'].str.split(r" to ", expand=True, n=0)[0].apply(lambda x: x.strip())
df['DEST_TO'] = df['DESTINATION'].str.split(r" to ", expand=True, n=0)[1].apply(lambda x: x.strip())
df.head()

,CARRIER,CDR_DATE,DIRECTION,DESTINATION,MINUTES,TOTAL_CALLS_CONNECTED,TOTAL_COST,MIN_RATE_PER_MIN,RATE_CURRENCY,TS_RECEIVED,DEST_FROM,DEST_TO
0,H_A1T T2 CFN,2021-04-01,Inbound,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to CHE,20.70,2,2.62890,0.127,Euro,2021-04-26 07-37-34,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE,CHE
1,H_A1T T2 CFN,2021-04-01,Inbound,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to IZH,5.22,2,0.66252,0.127,Euro,2021-04-26 07-37-34,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE,IZH
2,H_A1T T2 CFN,2021-04-01,Inbound,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to NIN,16.48,3,2.09338,0.127,Euro,2021-04-26 07-37-34,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE,NIN
3,H_A1T T2 CFN,2021-04-01,Inbound,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to PRM,3.68,1,0.46778,0.127,Euro,2021-04-26 07-37-34,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE,PRM
4,H_A1T T2 CFN,2021-04-01,Inbound,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE to SPB,0.08,2,0.01058,0.127,Euro,2021-04-26 07-37-34,AUSTRIA-A1 (TELEKOM AUSTRIA)-MOBILE,SPB


In [ ]:
#                     if not os.path.exists(filename):
#                         print(f'Downloading {filename}...')
#                         with open(filename, 'wb') as f:
#                             f.write(attachment.content)

In [ ]:
#                         msg_datetime_str = msg.datetime_received.replace(tzinfo=None).strftime("%Y-%m-%d %H-%M-%S")

In [6]:
for message in account.inbox.all():
    if message.attachments:
        for attachment in message.attachments:
            if isinstance(attachment, ItemAttachment):
                print(attachment.item.headers)

In [25]:
for message in account.inbox.all().order_by('-datetime_received')[:1]:
    for attachment in message.attachments:
        if isinstance(attachment, FileAttachment):
            local_path = os.path.join(attachment.name)
            with open(local_path, 'wb') as f:
                f.write(attachment.content)
            print('Saved attachment to', local_path)
        elif isinstance(attachment, ItemAttachment):
            if isinstance(attachment.item, Message):
                print(attachment.item.subject, attachment.item.body)
#     print(help(attachment.__dir__()))
#     print(message.content)

Saved attachment to 23321B3E.png
Saved attachment to 793516ab62f94a8f.jpg
Saved attachment to F7E026A3.png
